In [19]:
import numpy as np
import cv2
import os

import warnings
warnings.filterwarnings('ignore')

In [20]:
K=3
threshold=1
ITER_NUM = 2000

def residual_lengths(X, Y, s, t):
    e = np.dot(X, s) + Y
    diff_square = np.power(e - t, 2)
    residual = np.sqrt(np.sum(diff_square, axis=0))
    return residual
def ransac_fit(pts_s, pts_t):
    inliers_num = 0
    A = None
    t = None
    inliers = None
    for i in range(ITER_NUM):
        idx = np.random.randint(0, pts_s.shape[1], (K, 1))
        A_tmp, t_tmp = estimate_affine(pts_s[:, idx], pts_t[:, idx])
        residual = residual_lengths(A_tmp, t_tmp, pts_s, pts_t)
        if not(residual is None):
            inliers_tmp = np.where(residual < threshold)
            inliers_num_tmp = len(inliers_tmp[0])
            if inliers_num_tmp > inliers_num:
                inliers_num = inliers_num_tmp
                inliers = inliers_tmp
                A = A_tmp
                t = t_tmp
        else:
            pass
    return A, t, inliers

In [21]:
def estimate_affine(s, t):
    num = s.shape[1]
    M = np.zeros((2 * num, 6))
    
    for i in range(num):
        temp = [[s[0, i], s[1, i], 0, 0, 1, 0],
            [0, 0, s[0, i], s[1, i], 0, 1]]
        M[2 * i: 2 * i + 2, :] = np.array(temp)
    b = t.T.reshape((2 * num, 1))
    theta = np.linalg.lstsq(M, b)[0]
    X = theta[:4].reshape((2, 2))
    Y = theta[4:]
    return X, Y

In [22]:
def extract_SIFT(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, desc = sift.detectAndCompute(img_gray, None)
    kp = np.array([p.pt for p in kp]).T
    return kp, desc

def match_SIFT(descriptor_source, descriptor_target):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptor_source, descriptor_target, 
    k=2)
    pos = np.array([], dtype=np.int32).reshape((0, 2))
    matches_num = len(matches)
    for i in range(matches_num):
        if matches[i][0].distance <= 0.8 * matches[i][1].distance:
            temp = np.array([matches[i][0].queryIdx,
                            matches[i][0].trainIdx])
            pos = np.vstack((pos, temp))
    return pos

def affine_matrix(s, t, pos):
    s = s[:, pos[:, 0]]
    t = t[:, pos[:, 1]]
    _, _, inliers = ransac_fit(s, t)
    s = s[:, inliers[0]]
    t = t[:, inliers[0]]
    A, t = estimate_affine(s, t)
    M = np.hstack((A, t))
    return M

In [23]:
image1 = cv2.imread(fr"{os.getcwd()}\dataset\sister.JPG")
image2 = cv2.imread(fr"{os.getcwd()}\dataset\meAndSister.JPG")

In [24]:
def seeImage(image:"image read", name:str = "image"):
    cv2.imshow(name,image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [25]:
keypoint_source, descriptor_source = extract_SIFT(image1)
keypoint_target, descriptor_target = extract_SIFT(image2)

In [26]:
pos = match_SIFT(descriptor_source, descriptor_target)
H = affine_matrix(keypoint_source, keypoint_target, pos)

In [27]:
rows, cols, _ = image2.shape
warp = cv2.warpAffine(image1, H, (cols, rows))
merge = np.uint8(image2 * 0.5 + warp * 0.5)

In [18]:
seeImage(merge)